- 깁스 샘플링은 두 개 이상의 확률변수의 결합확률분포로부터 일련의 표본을 생성하는 확률적 알고리즘이다. 결합확률분포나 그에 관련된 확률 계산을 근사하기 위해 사용된다.
- 깁스 샘플링은 마코프 연쇄 몬테카를로 방법(Markov Chain Monte Carlo; MCMC) 알고리즘의 한 예이다.

- 몬테카를로 방법
    - 랜덤 표본을 뽑아 함수의 값을 확률적으로 계산하는 알고리즘이다.
    - 계산하려는 값이 닫힌 형식으로 표현되지 않거나 복잡한 경우에 그 값을 근사적으로 계산하려고 할 때 쓰인다.

- 마코프 연쇄
     - 마코프 가정을 따르는 이산 시간 확률 과정을 의미한다.
     - 특정 시점의 상태 확률은 단지 그 이전 상태에만 의존한다는 것이 핵심이다.
     - 즉, 한 상태에서 다른 상태로의 전이는 그동안 상태 전이에 대한 긴 이력을 필요로 하지 않고 바로 직전 상태에서의 전이로 추정할 수 있다는 이야기이다.
     - 특정 조건을 만족한 상태에서 마코프 연쇄를 반복하다 보면 현재 상태의 확률이 직전 상태의 확률과 같아지게, 수렴하게 된다. 이러한 평형 상태에 도달한 확률 분포를 정적분포라 한다.

- 마코프 연쇄 몬테카를로 방법
    - MCMC란 마코프 연쇄에 기반한 확률 분포로부터 표본을 추출하는 몬테카를로 방법이다.
    - 우리가 샘플을 얻으려고 하는 목표분포를 정적분포로 가지는 마코프 체인을 만든다.
    - 이 체인의 시뮬레이션을 가동하고 초기값에 영향을 받는 burn-in-period를 지나고 나면 목표 분포를 따르는 샘플이 만들어진다.

### 깁스 샘플링
- 깁스 샘플링은 MCMC의 일종이다.
- 몬테 카를로와 마코프 연쇄 몬테카를로의 차이점은 다음과 같다.
    - 몬테 카를로는 모든 샘플이 독립이고 생성될 확률 또한 랜덤이다.
    - 마코프 연쇄에 기반한 MCMC는 다음번 생성될 샘플은 현재 샘플의 영향을 받는다.
    - 깁스 샘플링은 다음번 생성될 표본은 현재 샘플에 영향을 받는다는 점에서 MCMC와 같지만, 나머지 변수는 그대로 두고 한 변수에만 변화를 준다는 점이 다르다.
        - 위의 말을 좀 더 자세히 해석하면 3개의 확률변수의 결합확률분포로부터 1개의 표본을 얻으려고 할 때,즉, p(x_1,x_2,x_3)가 있어서, 샘플 X = (x_1_0,x_2_0,_x_3_0)을 뽑게 되고, 새로운 표본을 뽑을 때, 각각의 확률 변수에서 나온 샘플들의 refresh가 진행되는 과정에서, 나머지 변수들을 고정시켜서 조건부 확률 분포로 만든다. 그렇게 3번을 진행하여 새로운 샘플을 만들어내는 것이다.
    - 새로운 값을 뽑는 데 쓰인 조건부 확률은 결합확률분포에 비례한다. 표본의 앞부분은 초기 상태에 의존하는(burn-in-period) 경향이 있지만, iteration이 높아지면 초기 상태에 관계 없이 p에 기반한 표본을 수집할 수 있다.

- 깁스 샘플링의 변형
    - 기존의 깁스 샘플링은 나머지 변수들을 고정시킨 상태에서 하나씩 업데이트를 수행한다.
- Block Gibbs sampling
    - 그룹으로 묶어 뽑는 기법이다. 즉, 하나씩 업데이트를 하는 것이 아니라, 2개 이상씩 변수들을 묶어서 업데이트한다.
- Collapsed Gibbs sampling
    - 불필요한 일부 변수를 샘플링에서 생략하는 기법이다. 예로 들어서 b가 그렇다면 c를 고정시킨 상태에서 a를 뽑고, a를 고정시킨 상태에서 c를 뽑는다.

In [1]:
import random

def roll_a_die():
    return random.choice(range(1,7))

In [2]:
def direct_sample():
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1,d1 + d2#결합 확률분포

In [4]:
def random_y_given_x(x):
    # x값을 알고 있다는 전제 하에
    # y값이 선택될 확률
    # y는 x+1, x+2, x+3
    # x+4, x+5, x+6 가운데 하나
    return x + roll_a_die()

In [7]:
def random_x_given_y(y):
    # y값을 알고 있다는 전제 하에
    # x값이 선택될 확률
    # 첫째 둘째 주사위 값의 합이 7이거나
    # 7보다 작다면
    if y <= 7:
        # 첫번째 주사위의 눈은 1~6
        # 각 눈이 선택될 확률은 동일
        return random.randrange(1, y)
    # 만약 총합이 7보다 크다면
    else:
        # 첫번째 주사위의 눈은
        # y-6, y-5,..., 6
        # 각 눈이 선택될 확률은 동일
        return random.randrange(y-6, 7)

In [5]:
def gibbs_sample(num_iters=100):
    # 초기값이 무엇이든 상관없음
    x, y = 1, 2
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

In [18]:
direct_sample() , gibbs_sample(10000000)

((5, 7), (4, 6))

- 깁스 샘플 수를 늘려서 결합 확률 분포로부터 뽑은 결과와 비교하면 유사한 결과가 나오는 것을 확인할 수 있다.
- 결합확률분포를 모를 때, 이미 알고있는 일부 조건부 확률분포에 깁스 샘플링을 적용하여 해당 결합확률분포의 표본을 얻어낼 수 있다.